In [1]:
import jax.random as jrn
import jax.numpy as jnp
import jax.lax as jla
import jax
import random

from pprint import pprint

from game.run import dummy_history

In [2]:
player_total = 9
game_len = 15

key = jrn.PRNGKey(random.randint(0, 1000))
history = dummy_history(key, player_total, game_len, prob_vote=0.7)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
t = random.randint(0, game_len)

print(f"{t=}")

state = {}

for k, v in history.items():
    state[k] = v[t]


state.keys()

t=10


dict_keys(['roles', 'presi', 'proposed', 'chanc', 'voted', 'tracker', 'draw', 'disc', 'presi_shown', 'chanc_shown', 'board', 'killed', 'winner'])

In [5]:
from bots.tokenize import tokenize

tokenized = tokenize(state)

size = 0

for k, v in tokenized.items():
    print(f"{k=}")
    print(f"{v.shape=}")
    print()

    size += v.shape[-1]


size

k='board'
v.shape=(9, 15, 11)

k='chanc'
v.shape=(9, 15, 10)

k='chanc_shown'
v.shape=(9, 15, 4)

k='killed'
v.shape=(9, 15, 9)

k='presi'
v.shape=(9, 15, 10)

k='presi_shown'
v.shape=(9, 15, 5)

k='proposed'
v.shape=(9, 15, 10)

k='roles'
v.shape=(9, 15, 27)

k='tracker'
v.shape=(9, 15, 3)

k='voted'
v.shape=(9, 15, 9)



98

In [104]:
def mask_roles(player: int, roles, **_):
    # roles do not change over time
    history = roles.shape[0]
    roles = roles[0]

    player_total = roles.shape[-1]
    mask_full = jnp.arange(player_total) == player

    role = roles[player] == 1
    mask_facist = roles == 1
    mask_facist |= roles == 2

    mask = jla.select(role, mask_facist, mask_full)
    masked =  roles * mask

    return jnp.tile(masked, (history, 1))


player = random.randint(0, player_total)
print("player", player)

role = state["roles"][0, player]
print("role", role)

masked_roles = mask_roles(player, **state)
print(masked_roles)

player 9
role 1
[[2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]
 [2 1 0 1 0 0 0 0 1]]


In [31]:
def mask_presi_shown(player: int, presi, presi_shown, **_):
    mask = player == presi
    return presi_shown * mask[:, None]
    
    
player = random.randint(0, player_total-1)
print(f"{player=}")

presi = state["presi"]
print(f"{presi=}")

masked_presi_shown = mask_presi_shown(player, **state)
print(f"{masked_presi_shown=}")

player=2
presi=Array([ 5,  4,  3,  1,  8,  7,  6,  5,  4,  3,  2,  1,  0, -1, -1], dtype=int32)
masked_presi_shown=Array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 2],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int32)


In [33]:
def mask_chanc_shown(player: int, chanc, chanc_shown, **_):
    mask = player == chanc
    return chanc_shown * mask[:, None]


player = random.randint(0, player_total-1)
print(f"{player=}")

chanc = state["chanc"]
print(f"{chanc=}")

masked_chanc_shown = mask_chanc_shown(player, **state)
print(f"{masked_chanc_shown=}")

player=8
chanc=Array([ 6,  1,  6,  6,  4,  7,  7,  1,  0,  6,  8,  2,  4, -1, -1], dtype=int32)
masked_chanc_shown=Array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 2],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int32)


In [48]:
def mask_state(player: int, state):
    masked = {}

    for k, v in state.items():
        if k in ["draw", "disc", "winner"]:
            continue

        elif k == "roles":
            masked[k] = mask_roles(player, **state)

        
        elif k == "presi_shown":
            masked[k] = mask_presi_shown(player, **state)

        elif k == "chanc_shown":
            masked[k] = mask_chanc_shown(player, **state)

        else:
            masked[k] = v

    return masked


player = random.randint(0, player_total-1)
print(f"{player=}")

# mask_state_vmap = jax.vmap(mask_state, in_axes=(0, None))
# players = jnp.arange(player_total)
# masked_state = mask_state_vmap(players, state)

masked_state = mask_state(player, state)

for k, v in masked_state.items():
    print(k)
    print(*v.astype(int))

player=1
roles
2 1 0 1 0 0 0 0 1
presi
5 4 3 1 8 7 6 5 4 3 2 1 0 -1 -1
proposed
6 1 8 6 4 8 7 1 0 6 8 2 4 -1 -1
chanc
6 1 6 6 4 7 7 1 0 6 8 2 4 -1 -1
voted
[0 1 0 1 0 1 1 1 1] [0 1 0 1 0 0 0 1 1] [0 1 0 0 1 0 0 0 1] [0 1 0 1 1 1 0 0 1] [0 1 0 0 1 1 0 1 1] [0 1 0 0 0 0 0 1 1] [0 1 0 1 1 0 1 1 0] [0 1 0 0 0 1 0 1 1] [1 0 0 0 1 1 1 1 1] [1 1 1 1 0 0 1 1 1] [1 0 1 1 1 1 1 1 1] [0 1 1 1 1 1 0 1 1] [0 0 0 1 1 1 1 0 1] [0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0]
tracker
0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
presi_shown
[0 0] [0 0] [0 0] [1 2] [0 0] [0 0] [0 0] [0 0] [0 0] [0 0] [0 0] [1 2] [0 0] [0 0] [0 0]
chanc_shown
[0 0] [2 0] [0 0] [0 0] [0 0] [0 0] [0 0] [2 0] [0 0] [0 0] [0 0] [0 0] [0 0] [0 0] [0 0]
board
[4 7] [3 7] [2 7] [2 7] [2 6] [1 6] [1 6] [1 5] [0 5] [0 4] [0 3] [0 2] [0 1] [0 0] [0 0]
killed
[1 0 1 0 0 0 0 0 0] [1 0 1 0 0 0 0 0 0] [1 0 1 0 0 0 0 0 0] [1 0 1 0 0 0 0 0 0] [1 0 1 0 0 0 0 0 0] [1 0 1 0 0 0 0 0 0] [1 0 1 0 0 0 0 0 0] [1 0 1 0 0 0 0 0 0] [1 0 1 0 0 0 0 0 0] [0 0 1 0 0 0 0 0 0]

In [59]:
def one_hot(x, maxval, minval=0):
    return jnp.eye(maxval - minval)[x - minval]


def roles_tokenize(roles, **_):
    return one_hot(roles, 3)


print(masked_state["roles"])
print(roles_tokenize(**masked_state).astype(int))

[2 1 0 1 0 0 0 0 1]
[[0 0 1]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]]


In [109]:
def presi_tokenize(presi, roles, **_):
    player_total = roles.shape[-1]
    return one_hot(presi, player_total, -1)


print(masked_state["presi"])
print(presi_tokenize(**masked_state).astype(int))

[ 5  4  3  1  8  7  6  5  4  3  2  1  0 -1 -1]
[[0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]]


In [110]:
def proposed_tokenize(proposed, roles, **_):
    player_total = roles.shape[-1]
    return one_hot(proposed, player_total, -1)


print(masked_state["proposed"])
print(proposed_tokenize(**masked_state).astype(int))

[ 6  1  8  6  4  8  7  1  0  6  8  2  4 -1 -1]
[[0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]]


In [63]:
def chanc_tokenize(chanc, roles, **_):
    player_total = roles.shape[-1]
    return one_hot(chanc, player_total, -1)


print(masked_state["chanc"])
print(chanc_tokenize(**masked_state).astype(int))

[ 6  1  6  6  4  7  7  1  0  6  8  2  4 -1 -1]
[[0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]]


In [64]:
def voted_tokenize(voted, **_):
    return voted


print(masked_state["voted"])
print(voted_tokenize(**masked_state).astype(int))

[[False  True False  True False  True  True  True  True]
 [False  True False  True False False False  True  True]
 [False  True False False  True False False False  True]
 [False  True False  True  True  True False False  True]
 [False  True False False  True  True False  True  True]
 [False  True False False False False False  True  True]
 [False  True False  True  True False  True  True False]
 [False  True False False False  True False  True  True]
 [ True False False False  True  True  True  True  True]
 [ True  True  True  True False False  True  True  True]
 [ True False  True  True  True  True  True  True  True]
 [False  True  True  True  True  True False  True  True]
 [False False False  True  True  True  True False  True]
 [False False False False False False False False False]
 [False False False False False False False False False]]
[[0 1 0 1 0 1 1 1 1]
 [0 1 0 1 0 0 0 1 1]
 [0 1 0 0 1 0 0 0 1]
 [0 1 0 1 1 1 0 0 1]
 [0 1 0 0 1 1 0 1 1]
 [0 1 0 0 0 0 0 1 1]
 [0 1 0 1 1 0 1 1 

In [65]:
def tracker_tokenize(tracker, **_):
    return one_hot(tracker, 3)


print(masked_state["tracker"])
print(tracker_tokenize(**masked_state).astype(int))

[0 0 1 0 0 1 0 0 0 0 0 0 0 0 0]
[[1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]]


In [81]:
def presi_shown_tokenize(presi_shown, **_):
    return jnp.array([
        # empty
        presi_shown.sum(axis=-1) == 0,
        # number of F policies
        presi_shown[:, 0] == 3, # 0
        presi_shown[:, 1] == 1, # 1
        presi_shown[:, 1] == 2, # 2
        presi_shown[:, 1] == 3, # 3
    ]).T


print(masked_state["presi_shown"])
print(presi_shown_tokenize(**masked_state).astype(int))

[[0 0]
 [0 0]
 [0 0]
 [1 2]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 2]
 [0 0]
 [0 0]
 [0 0]]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 1 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]]


In [82]:
def chanc_shown_tokenize(chanc_shown, **_):
    return jnp.array([
        # empty
        chanc_shown.sum(axis=-1) == 0,
        # number of F policies
        chanc_shown[:, 0] == 2, # 0
        chanc_shown[:, 1] == 1, # 1
        chanc_shown[:, 1] == 2, # 2
    ]).T


print(masked_state["chanc_shown"])
print(chanc_shown_tokenize(**masked_state).astype(int))

[[0 0]
 [2 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [2 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]
[[1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]


In [83]:
def board_tokenize(board, **_):
    board_1 = board[:, 0]
    board_1 = one_hot(board_1, 5)
    
    board_2 = board[:, 1]
    board_2 = one_hot(board_2, 6)

    return jnp.concatenate([board_1, board_2], axis=-1)


print(masked_state["board"])
print(board_tokenize(**masked_state).astype(int))

[[4 7]
 [3 7]
 [2 7]
 [2 7]
 [2 6]
 [1 6]
 [1 6]
 [1 5]
 [0 5]
 [0 4]
 [0 3]
 [0 2]
 [0 1]
 [0 0]
 [0 0]]
[[0 0 0 0 1 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 0 0]]


In [108]:
def killed_tokenize(killed, **_):
    return killed.astype(int)


print(masked_state["killed"])
print(killed_tokenize(**masked_state).astype(int))

[[ True False  True False False False False False False]
 [ True False  True False False False False False False]
 [ True False  True False False False False False False]
 [ True False  True False False False False False False]
 [ True False  True False False False False False False]
 [ True False  True False False False False False False]
 [ True False  True False False False False False False]
 [ True False  True False False False False False False]
 [ True False  True False False False False False False]
 [False False  True False False False False False False]
 [False False False False False False False False False]
 [False False False False False False False False False]
 [False False False False False False False False False]
 [False False False False False False False False False]
 [False False False False False False False False False]]
[[1 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 